#### Sigmoid

In [8]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [9]:
# print(sigmoid(1))

0.7310585786300049


In [10]:
# print(sigmoid(-1000))

0.0


C:\Users\dlarh\anaconda3\envs\gan\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [11]:
%run chapter1.ipynb

In [12]:
def pulsar_exec(epoch_count = 10, mb_size = 10, report = 1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [13]:
def load_pulsar_dataset():
    with open('data/chap02/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype = 'float32')

### 순전파, 역전파 함수 정의

In [14]:
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output) #두 원소에 대한 기스모이드 교체 엔트로피 값 계산
    #logit값으로 계산하기 때문에 시그모이드 함수를 거치지 않음
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

In [15]:
def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape) #G_loss값을 행렬의 원소수로 나누어 각 원소의 손실 기울기로 부여
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output) #y와 output 의 각 원소쌍에 대해 
    #시그모이드 교차 엔트로피의 편미분 값을 구해 entropy와 output사이의 부분기울기로 함
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

### 정확도 계산 함수

In [16]:
def eval_accuracy(output, y):
    estimate = np.greater(output, 0) #output에 담긴 로짓값의 부호를 통해 결과 판단
    #로짓값을 확률로 변환하는 시그모이드 함수 필요 X
    answer = np.greater(y, 0.5 ) 
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

### Sigmoid 함수 정의

In [17]:
def relu(x):
    return np.maximum(x, 0) #np.max 함수는 배열과 숫자0을 비교, np.maximun 함수는 각 원소를 0과 비교

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def sigmoid_derv(x, y):
    return y * (1-y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)